# World Cities | Weather Analysis

### As illustrated by the temperature plot, cities closer to the equator (latitude 0) have higher temperatures, while cities approaching the outer poles have low temperatures.

### There does not appear to be any significant correlation between cloudiness and proximity to the equator. In fact, the cloudiness plot illustrates a wide variety of cloudiness levels among  world cities.

### There does not appear to be a strong correlation between wind speed and proximity to the equator. However, most of the world cities sampled tend to have low to medium wind speeds. A few outliers are characterized with high wind speeds.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from pprint import pprint

# Import API key
from config import weather_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [ ]:
# create cities dataframe
cities_df = pd.DataFrame({"City": cities})
cities_df.head()

In [ ]:
# set up additional columns to hold information
cities_df['Temp'] = ""
cities_df['Humidity'] = ""
cities_df['Cloudiness'] = ""
cities_df['Wind Speed'] = ""
cities_df['Latitude'] = ""

cities_df.head()

## Perform API Calls

In [ ]:
# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

query_url = f"{url}appid={weather_key}&units={units}&q=" 
query_url


In [ ]:
# # query url needs a city at the end
# # test with Paris

# parisurl = query_url+"Paris"
# parisurl

# parisresp = requests.get(parisurl).json()
# pprint(parisresp)

# paristemp = parisresp["main"]["temp"]
# parishumid = parisresp["main"]["humidity"]
# parisclouds = parisresp["clouds"]["all"]
# pariswinds = parisresp["wind"]["speed"]

# print(paristemp)
# print(parishumid)
# print(parisclouds)
# print(pariswinds)



In [ ]:
temps = []
humidity = []
clouds = []
wind = []
lati = []

print("Beginning Data Retrieval")
print("---------------------------------")

# use iterrows to iterate through pandas dataframe
for index, row in cities_df.iterrows():

    # get city from df
    city = row['City']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    print(f"from: {query_url}{city}")
    response = requests.get(query_url+city).json()
    
#     # extract results
#     results = response['results']
    
    try:
                
        cities_df.loc[index, 'Temp'] = float(response["main"]["temp"])
        cities_df.loc[index, 'Humidity'] = float(response["main"]["humidity"])
        cities_df.loc[index, 'Cloudiness'] = float(response["clouds"]["all"])
        cities_df.loc[index, 'Wind Speed'] = float(response["wind"]["speed"])
        cities_df.loc[index, 'Latitude'] = float(response["coord"]["lat"])
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
len(cities_df)
pd.to_numeric(cities_df["Temp"], errors = "ignore")
pd.to_numeric(cities_df["Latitude"], errors = "ignore")
pd.to_numeric(cities_df["Cloudiness"], errors = "ignore")
pd.to_numeric(cities_df["Wind Speed"], errors = "ignore")
cities_df.head()

In [ ]:
cities_df.dtypes

In [ ]:
# # Build a scatter plot for each data type
plt.scatter(cities_df["Latitude"], cities_df["Temp"], marker="o", color = "pink")

# # Incorporate the other graph properties
plt.title("Temperature in World Cities (10/1/18)")
plt.ylabel("Temperature")
plt.xlabel("Latitude")
plt.grid(True)

plt.savefig("TemperatureInWorldCities.png")

In [ ]:
# # Build a scatter plot for each data type
plt.scatter(cities_df["Latitude"], cities_df["Humidity"], marker="o", color = "teal")

# # Incorporate the other graph properties
plt.title("Humidity in World Cities (10/1/18)")
plt.ylabel("Humidity")
plt.xlabel("Latitude")
plt.grid(True)

plt.savefig("HumidityInWorldCities.png")

In [ ]:
# # Build a scatter plot for each data type
plt.scatter(cities_df["Latitude"], cities_df["Cloudiness"], marker="o", color = "skyblue")

# # Incorporate the other graph properties
plt.title("Cloudiness in World Cities (10/1/18)")
plt.ylabel("Cloudiness")
plt.xlabel("Latitude")
plt.grid(True)

plt.savefig("CloudinessInWorldCities.png")

In [ ]:
# # Build a scatter plot for each data type
plt.scatter(cities_df["Latitude"], cities_df["Wind Speed"], marker="o", color = "orange")

# # Incorporate the other graph properties
plt.title("Wind Speed in World Cities (10/1/18)")
plt.ylabel("Wind Speed")
plt.xlabel("Latitude")
plt.grid(True)

plt.savefig("WindSpeedInWorldCities.png")

In [ ]:
# Save csv file of dataframe
cities_df.to_csv("./WeatherPy.csv")